In [1]:
# add to path
import sys
if '../' not in sys.path:
    sys.path.insert(0, '../')
sys.path

['../',
 '/home/jimmyyao/repos/rev-stable-diffusion-2/notebooks',
 '/home/jimmyyao/miniconda3/envs/rev-sd2/lib/python39.zip',
 '/home/jimmyyao/miniconda3/envs/rev-sd2/lib/python3.9',
 '/home/jimmyyao/miniconda3/envs/rev-sd2/lib/python3.9/lib-dynload',
 '',
 '/home/jimmyyao/.local/lib/python3.9/site-packages',
 '/home/jimmyyao/miniconda3/envs/rev-sd2/lib/python3.9/site-packages',
 '/home/jimmyyao/repos/segment-anything',
 '/home/jimmyyao/miniconda3/envs/rev-sd2/lib/python3.9/site-packages/certifi-2022.12.7-py3.9.egg',
 '/home/jimmyyao/miniconda3/envs/rev-sd2/lib/python3.9/site-packages/wheel-0.40.0-py3.9.egg',
 '/home/jimmyyao/miniconda3/envs/rev-sd2/lib/python3.9/site-packages/setuptools-67.6.1-py3.9.egg',
 '/home/jimmyyao/repos/GenerativeImage2Text']

In [2]:
# param = {}

# image_encoder = get_image_encoder(
#         param.get('image_encoder_type', 'CLIPViT_B_16'),
#         input_resolution=param.get('test_crop_size', 224),
#     )

In [3]:
# i1 = torch.randn(2, 3, 120, 120)

# image_encoder(i1).shape

In [4]:
# t1 = torch.rand(768)
# t2 = torch.rand(768)
# t3 = torch.rand(768)

# li = [t1, t2, t3]
# # convert li to tensor
# t = torch.stack(li)
# t.shape

In [5]:

from generativeimage2text.train import (get_image_transform, get_transform_image_norm, 
                                        get_inception_train_transform, 
                                        get_data, collate_fn, recursive_to_device)
from generativeimage2text.common import Config

from transformers import BertTokenizer

import torch
import logging

from segment_anything import sam_model_registry, SamAutomaticMaskGenerator
from torch.cuda.amp import autocast
    
from samgit.model import sam_image_crops, get_samgit_model

import cv2
import torchvision.transforms as transforms
from PIL import Image


def get_small_transform_vit_default(cfg, crop=24):
    default_normalize = transforms.Normalize(
            mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    normalize = get_transform_image_norm(cfg, default_normalize)
    transform = get_inception_train_transform(
        bgr2rgb=True,
        crop_size=crop,
        normalize=normalize,
        small_scale=cfg.input_small_scale,
        no_color_jitter=cfg.no_color_jitter,
        no_flip=cfg.no_flip,
        no_aspect_dist=cfg.no_aspect_dist,
        resize_crop=cfg.resize_crop,
        max_size=cfg.train_max_size,
        interpolation=cfg.interpolation or Image.BILINEAR,
    )
    return transform

def prepare_model():
    cfg = {
        'crop_region_extend_in_datatransform': 4,
        'data_normalize': 'clip',
        'train_crop_size': 224,
        'input_small_scale': 0.8,
        'no_color_jitter': True,
        'no_flip': True,
        'no_aspect_dist': True,
        'interpolation': 'bicubic',
        'min_size_range32': [160, 224], # in pretraining, it is multi-scale from 160 to 224; while for fine-tuning, it is single scale
        # 'min_size_range32': [224], # in pretraining, it is multi-scale from 160 to 224; while for fine-tuning, it is single scale
        'patch_size': 16,
        'train_transform': 'vitp',
    }
    cfg = Config(cfg, {})
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
    
    param = {
        "top_n_bbox": 4,
    }
    model = get_samgit_model(tokenizer, param)
    model.half()
    model.train()
    model.cuda()
    
    param = {
        "top_n_bbox": 4,
    }
    
    # preprocess object crops with sam
    sam = sam_model_registry[param.get("sam_model_type", "vit_h")](checkpoint=param.get("sam_checkpoint", "../models/sam_vit_h_4b8939.pth")).half()
    sam.half()
    sam.eval()
    sam.requires_grad_(False)
    sam.to(device='cuda')
    mask_generator = SamAutomaticMaskGenerator(
        model=sam,
        points_per_side=4,
        pred_iou_thresh=0.95,
        stability_score_thresh=0.95,
    )
    
    return model, mask_generator, cfg, param

def forward_backward(model, mask_generator, cfg, param, image_files, captions, prefixs=None):
    if prefixs is None:
        prefixs = [''] * len(captions)

    all_data = []
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
    
    image_transform = get_image_transform(cfg)
    # resize to 24x24
    small_transform = get_small_transform_vit_default(cfg, crop=24)
    
    for image_file, prefix, target in zip(image_files, prefixs, captions):
        data = get_data(image_file, prefix, target,
                        tokenizer, image_transform)
        all_data.append(data)
    data = collate_fn(all_data)
    data = recursive_to_device(data, 'cuda')
    
    logging.info("cuda util: %s", torch.cuda.utilization())
    with autocast():
        batch_img_crops = []
        for img_name in image_files:
            image = cv2.imread(img_name)
            crops = sam_image_crops(image, mask_generator, param.get('top_n_bbox', 4))
            new_crops = []
            for crop in crops:
                if 0 in crop.shape:
                    continue
                crop = Image.fromarray(crop)
                new_crops.append(small_transform(crop))

            if len(new_crops) == 0:
                batch_img_crops.append(None)
            else:
                crops = torch.stack(new_crops)
                crops.half()
                batch_img_crops.append(crops)
            
        loss_dict = model(data, batch_img_crops) # hard-coded to cuda
        
    loss = sum(loss_dict.values())
    loss.backward()
    logging.info(loss)
    
    return loss


/home/jimmyyao/miniconda3/envs/rev-sd2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# get data
import os
import pandas as pd
DIFFUSIONDB_IMGS_PATH = "/mnt/d/AiStuff/data/diffusionDB2M-f2/images"

# # csvs
# csvs = ["diffusiondb-filtered-0-40000.csv", "diffusiondb-filtered-40001-80000.csv", "diffusiondb-filtered-80001-120000.csv", "diffusiondb-filtered-120001-144894.csv"]
# dfs = []
# for csv in csvs:
#     df = pd.read_csv(os.path.join(DIFFUSIONDB_IMGS_PATH, csv))
#     dfs.append(df)
# df = pd.concat(dfs)

# df.reset_index(drop=True, inplace=True)
# df.to_csv(os.path.join(DIFFUSIONDB_IMGS_PATH, "diffusiondb-filtered.csv"), index=False)
df = pd.read_csv(os.path.join(DIFFUSIONDB_IMGS_PATH, "diffusiondb-filtered.csv"))
df

,image_name,prompt,part,new_image_name
0,/kaggle/input/diffusiondb-2m-part-0301-to-0400...,bbc article announcing the beginning of a nucl...,0,diffusiondb-filtered-0-40000/b935f1f4-00fe-47f...
1,/kaggle/input/diffusiondb-2m-part-1301-to-1400...,haunted house ghost bugs dark room grey black,0,diffusiondb-filtered-0-40000/f3c98c88-d89c-42b...
2,/kaggle/input/diffusiondb-2m-part-0601-to-0700...,la llorona serving cunt and being fierce,0,diffusiondb-filtered-0-40000/8d4dd42f-846f-4e7...
3,/kaggle/input/diffusiondb-2m-part-0501-to-0600...,"Medusa, fantasy horror art, hissing scream, fa...",0,diffusiondb-filtered-0-40000/c67d39f8-98f4-439...
4,/kaggle/input/diffusiondb-2m-part-0101-to-0200...,cleopatra in ancient egypt in front of a pyram...,0,diffusiondb-filtered-0-40000/b5141b36-447c-492...
...,...,...,...,...
144890,/kaggle/input/diffusiondb-2m-part-0701-to-0800...,racionais mc's com traje de astronauta,3,diffusiondb-filtered-120001-144894/546518b9-74...
144891,/kaggle/input/diffusiondb-2m-part-0801-to-0900...,dante traveling through the 7 layers of hell,3,diffusiondb-filtered-120001-144894/02ceb3f5-30...
144892,/kaggle/input/diffusiondb-2m-part-1801-to-1900...,"Spike Spiegel by Kentaro Miura, Charachter Por...",3,diffusiondb-filtered-120001-144894/21f3563b-6d...
144893,/kaggle/input/diffusiondb-2m-part-1301-to-1400...,can't sleep. clown will eat me. horror ; night...,3,diffusiondb-filtered-120001-144894/8f13247b-55...


In [7]:
len(os.listdir(os.path.join(DIFFUSIONDB_IMGS_PATH, "diffusiondb-filtered-0-40000")))

40001

In [8]:
# first 100 for eval
df_eval = df[:100]
# rest for training
df_train = df[100:]

In [9]:
import logging
logging.basicConfig(filename='example.log', encoding='utf-8', level=logging.DEBUG)

In [10]:

import numpy as np
import time
from tqdm import tqdm

# 1 epoch
model, mask_generator, cfg, param = prepare_model()
model.train()

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)


start = time.time()
train_losses = []
pbar = tqdm(np.array_split(df_train, (len(df)//16) + 1))
for chunk in pbar:
    image_files = chunk['new_image_name'].tolist()
    image_files = [os.path.join(DIFFUSIONDB_IMGS_PATH, image_file) for image_file in image_files]
    captions = chunk['prompt'].tolist()
    
    loss = forward_backward(model, mask_generator, cfg, param, image_files, captions)
    optimizer.step()
    
    optimizer.zero_grad()
    pbar.set_description(f"[{pbar.n}] loss: {loss:.4f}")
    train_losses.append(loss)
    
end = time.time()

print("EVAL")
model.eval()
eval_losses = []
for i, chunk in enumerate(np.array_split(df_eval, (len(df)//16) + 1)):
    
    image_files = chunk['new_image_name'].tolist()
    captions = chunk['prompt'].tolist()
    
    loss = forward_backward(model, mask_generator, cfg, param, image_files, captions)
    eval_losses.append(loss)
    

[5] loss: nan:   0%|          | 6/9056 [00:31<13:22:54,  5.32s/it]   


KeyboardInterrupt: 

In [ ]:
# save model dict and loss

torch.save({
            'epoch': 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            }, f"checkpoint-{1}.pt")

In [ ]:
import pandas as pd
prompt_df = pd.read_csv('../datasets/sampleeval/prompts.csv')
image_files = [f"../datasets/sampleeval/{img}.png" for img in prompt_df['imgId']]
captions = [cap for cap in prompt_df['prompt']]

print(image_files)
print(captions)

['../datasets/sampleeval/20057f34d.png', '../datasets/sampleeval/227ef0887.png', '../datasets/sampleeval/92e911621.png', '../datasets/sampleeval/a4e1c55a9.png', '../datasets/sampleeval/c98f79f71.png', '../datasets/sampleeval/d8edf2e40.png', '../datasets/sampleeval/f27825b2c.png']
['hyper realistic photo of very friendly and dystopian crater', 'ramen carved out of fractal rose ebony, in the style of hudson river school', 'ultrasaurus holding a black bean taco in the woods, near an identical cheneosaurus', 'a thundering retro robot crane inks on parchment with a droopy french bulldog', 'portrait painting of a shimmering greek hero, next to a loud frill-necked lizard', 'an astronaut standing on a engaging white rose, in the midst of by ivory cherry blossoms', 'Kaggle employee Phil at a donut shop ordering all the best donuts, with a speech bubble that proclaims "Donuts. It\'s what\'s for dinner!"']


In [ ]:
# forward_backward(image_files, captions)

cuda util: 94
image_features torch.Size([7, 101, 768])
